# Bernoulli Console With Random Orientation and Global Distributed Loads

In [7]:
L, d, t = 1000.0, 100.0, 5.0
Ex, nu = 210000.0, 0.25

In [8]:
from linkeddeepdict.tools import getallfromkwargs
from neumann.linalg import Vector, linspace
from neumann import repeat
from polymesh.space import StandardFrame, PointCloud
from polymesh.utils.space import index_of_closest_point, index_of_furthest_point
from polymesh.utils.topology import L2_to_L3
from sigmaepsilon.fem.cells import B2, B3
from sigmaepsilon import Structure, LineMesh, PointData, BeamSection
import numpy as np
from latexdocs.utils import floatformatter

seed = 0  # integer or None
seed = None

rs = np.random.RandomState()
if isinstance(seed, int):
    rs.seed(seed)

formatter = floatformatter(sig=4)
def f2s(x): return formatter.format(x)


# section
section = BeamSection('CHS', d=d, t=t, n=32)
section.calculate_section_properties()
section_props = section.section_properties
A, Ix, Iy, Iz = getallfromkwargs(['A', 'Ix', 'Iy', 'Iz'], **section_props)


# material
G = Ex / (2 * (1 + nu))
Hooke = np.array([
    [Ex*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, Ex*Iy, 0],
    [0, 0, 0, Ex*Iz]
])


def solve(n, angles, loads, celltype=B2):
    # space
    GlobalFrame = StandardFrame(dim=3)
    TargetFrame = GlobalFrame.rotate('Body', angles, 'XYZ', inplace=False)

    # mesh
    p0 = np.array([0., 0., 0.])
    p1 = np.array([L, 0., 0.])
    coords = linspace(p0, p1, n+1)
    points = PointCloud(coords, frame=TargetFrame)
    coords = points.show(GlobalFrame)
    topo = np.zeros((n, 2), dtype=int)
    topo[:, 0] = np.arange(n)
    topo[:, 1] = np.arange(n) + 1
    if celltype.NNODE == 3:
        coords, topo = L2_to_L3(coords, topo)
    i_first = index_of_closest_point(coords, np.array([0., 0., 0.]))
    i_last = index_of_furthest_point(coords, np.array([0., 0., 0.]))

    # essential boundary conditions
    penalty = 1e40  # penalty value
    fixity = np.zeros((coords.shape[0], 6)).astype(bool)
    fixity[i_first, :] = True
    fixity = fixity.astype(float) * penalty

    # natural boundary conditions
    loads = np.array(loads)
    loads[:3] = Vector(loads[:3], frame=GlobalFrame).show(TargetFrame)
    nodal_loads = np.zeros((coords.shape[0], 6))
    cell_loads = np.zeros((topo.shape[0], topo.shape[1], 6))
    cell_loads[:, :, :] = loads

    # pointdata
    pd = PointData(coords=coords, frame=GlobalFrame,
                   loads=nodal_loads, fixity=fixity)

    # celldata
    frames = repeat(TargetFrame.axes, topo.shape[0])
    cd = celltype(topo=topo, loads=cell_loads, material=Hooke, frames=frames)

    # set up mesh and structure
    mesh = LineMesh(pd, cd, frame=GlobalFrame)
    structure = Structure(mesh=mesh)

    structure.linear_static_analysis()
    structure.nodal_dof_solution()
    reactions = structure.reaction_forces()
    return reactions[i_first]


In [9]:
loads = [0, 0, 1.0, 0, 0, 0]
n = 2  # number of elements
cells = [B2, B3]

for cell in cells:
    for _ in range(10):
        angles = np.random.rand(3) * np.pi * 2
        r = solve(n, angles, loads, cell)
        print([f2s(v) for v in r[:3]])

['1.085e-11', '-4.908e-12', '-1000']
['-7.538e-13', '1.442e-13', '-1000']
['-8.085e-12', '2.411e-12', '-1000']
['9.557e-12', '-1.225e-13', '-1000']
['-1.791e-12', '-2.492e-12', '-1000']
['2.498e-13', '-3.969e-12', '-1000']
['2.224e-12', '-1.427e-12', '-1000']
['2.799e-12', '3.469e-13', '-1000']
['-1.287e-12', '2.792e-12', '-1000']
['-1.75e-12', '-1.129e-11', '-1000']
['-2.705e-12', '5.367e-11', '-1000']
['-9.311e-12', '9.85e-11', '-1000']
['-4.126e-12', '-6.698e-11', '-1000']
['-2.581e-12', '3.693e-13', '-1000']
['-9.388e-12', '-1.062e-11', '-1000']
['2.071e-11', '1.958e-11', '-1000']
['-6.725e-11', '1.35e-11', '-1000']
['2.531e-13', '-9.214e-11', '-1000']
['1.587e-11', '-3.235e-11', '-1000']
['-2.325e-12', '8.873e-13', '-1000']


In [10]:
def solve(n, angles, loads, celltype=B2):
    # space
    GlobalFrame = StandardFrame(dim=3)
    angles[1:] = 0.
    TargetFrame = GlobalFrame.rotate('Body', angles, 'XYZ', inplace=False)

    # mesh
    p0 = np.array([0., 0., 0.])
    p1 = np.array([L, 0., 0.])
    coords = linspace(p0, p1, n+1)
    topo = np.zeros((n, 2), dtype=int)
    topo[:, 0] = np.arange(n)
    topo[:, 1] = np.arange(n) + 1
    if celltype.NNODE == 3:
        coords, topo = L2_to_L3(coords, topo)
    i_first = index_of_closest_point(coords, np.array([0., 0., 0.]))
    i_last = index_of_furthest_point(coords, np.array([0., 0., 0.]))

    # essential boundary conditions
    penalty = 1e40  # penalty value
    fixity = np.zeros((coords.shape[0], 6)).astype(bool)
    fixity[i_first, :] = True
    fixity = fixity.astype(float) * penalty

    # natural boundary conditions
    loads = np.array(loads)
    loads[:3] = Vector(loads[:3], frame=GlobalFrame).show(TargetFrame)
    nodal_loads = np.zeros((coords.shape[0], 6))
    cell_loads = np.zeros((topo.shape[0], topo.shape[1], 6))
    cell_loads[:, :, :] = loads

    # pointdata
    pd = PointData(coords=coords, frame=GlobalFrame,
                   loads=nodal_loads, fixity=fixity)

    # celldata
    frames = repeat(TargetFrame.axes, topo.shape[0])
    cd = celltype(topo=topo, loads=cell_loads, material=Hooke, frames=frames)

    # set up mesh and structure
    mesh = LineMesh(pd, cd, frame=GlobalFrame)
    structure = Structure(mesh=mesh)

    structure.linear_static_analysis()

    dofsol = structure.nodal_dof_solution()
    
    reactions = structure.reaction_forces()
    return reactions[i_first]

In [11]:
loads = [0, 0, 1.0, 0, 0, 0]
n = 2  # number of elements
cells = [B2, B3]

for cell in cells:
    for _ in range(10):
        angles = np.random.rand(3) * np.pi * 2
        r = solve(n, angles, loads, cell)
        print([f2s(v) for v in r[:3]])

['0', '-7.105e-15', '-1000']
['0', '5.684e-14', '-1000']
['0', '-5.684e-14', '-1000']
['0', '3.787e-29', '-1000']
['0', '-1.776e-15', '-1000']
['0', '-5.684e-14', '-1000']
['0', '-5.684e-14', '-1000']
['0', '-5.684e-14', '-1000']
['0', '8.078e-28', '-1000']
['0', '-2.019e-28', '-1000']
['0', '-2.703e-12', '-1000']
['0', '2.315e-11', '-1000']
['0', '1.661e-12', '-1000']
['0', '-2.018e-11', '-1000']
['0', '4.768e-12', '-1000']
['0', '2.402e-11', '-1000']
['0', '-9.603e-13', '-1000']
['0', '-5.296e-13', '-1000']
['0', '-3.106e-13', '-1000']
['0', '-1.033e-11', '-1000']
